In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

3031


In [4]:
c = 0
for i in range(len(rgis_to_analyze)):
#     if c>0: continue
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]
    
    # if ga<500, we are not going to do any coarsening
    if ga<500:
        continue

    # choose how much to coarsen (more coarse for bigger glaciers) 
    if ga>1000:
        scale=5
    else:
        scale=3
    
    # set folder
    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
       
    # check if this glacier has been run already, skip if so
    temp_path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs_shadowed_coarse.nc")
#     if os.path.exists(temp_path):
#         continue
    
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
#     continue
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
 
    # open glacier mask, count how many pixels there are
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')
    glacier_pixels = glacier_mask.sum().values
    print(glacier_pixels)
    
#     # open the observed faction df
#     obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
#     obs_df = pd.read_csv(obs_path)
    
    # for each year, open the daily data, coarsen and resave
    for y in [2018,2019,2020,2021,2022]:
        print(y)
        
        # open data
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed.nc")
        
        # if small glacier, we dont need to chunk
        if ga>500:
            snow = xr.open_dataset(path_open, chunks={'time':1})
        else:
            snow = xr.open_dataset(path_open, chunks={'time':10})

        # make 1=snow, 0=ablation, nan=cloud,shadow,off-glacier
        snow = snow.where(snow!=0, np.nan).where(snow<=1, 0)
#         print(snow.dims)
        
        # coarsen snow product
        snow = snow.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True)
#         print(snow.dims)

        # fix to 0(nodata), 1(ablation), 2(snow)
        snow = xr.where(snow.isnull(), 0, xr.where(snow>=0.5, 2, 1)).astype('uint8')
        
        # reapply mask
        snow = snow.where(glacier_mask>0.5,0).astype('uint8')
        
        # save
        path_temp = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed_coarse.nc")
        encoding = {"class":{"zlib": True, "dtype": "uint8"}}
        snow.to_netcdf(path_temp, encoding=encoding)
     
     # then go through and calculate the percent useable on each date
#     print("Calculating percents")
#     for y in [2018,2019,2020,2021,2022]:

#         # open
#         path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed_coarse.nc")
#         snow = xr.open_dataset(path, chunks={'band':1, 'y':-1, 'x':-1}).to_array()
        
#         # get dates
#         dates = snow.time.values
#         dates = [str(d)[:10] for d in dates]
        
#         # count usable fraction each date
#         percent_usable_by_time = (xr.where(snow>0, 1, 0).sum(dim=['x','y'])).compute().values[0]
# #         print()
        
#         # scale correctly
#         percent_usable_by_time = percent_usable_by_time/glacier_pixels #*scale*scale
# #         print(len(percent_usable_by_time))
        
#         # add to obs_df
#         obs_df.loc[obs_df['Date'].isin(dates), 'observed_initial'] = percent_usable_by_time.round(4)
#         obs_df.to_csv(obs_path, index=False)
        
#     c+=1
print("Done!")


Starting 3010 of 3031: RGI60-01.17423  512.357 km2
576980
2018


C:\Users\lzell\Anaconda3\envs\AGVA_env\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


2019


C:\Users\lzell\Anaconda3\envs\AGVA_env\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


2020


...

2021


C:\Users\lzell\AppData\Local\Temp\ipykernel_3084\1508650737.py:69: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  snow = snow.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True)
C:\Users\lzell\Anaconda3\envs\AGVA_env\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


2022


C:\Users\lzell\AppData\Local\Temp\ipykernel_3084\1508650737.py:69: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  snow = snow.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True)
C:\Users\lzell\Anaconda3\envs\AGVA_env\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


Done!
